## 앙몬드 팀 모델 개발 설명서

안녕하세요. 저희는 과거 주식 보유 이력을 바탕으로 전체 보유 기간을 예측하고자 하였습니다. 주식 보유 기간에 영향을 주는 것이 무엇이 있을지 탐구해보면서, 가지고 있는 데이터를 통해 어떠한 정보를 도출할 수 있을지 고민하였습니다. 

## 1. 예측 모델에 대한 개발 설명 

### A. 분석 배경 및 과정

주식 보유 기간을 결정짓는 요인을 알기위해 배경조사를 우선 실시하였습니다. 자본시장연구원에서 나온 [주식시장 개인투자자의 행태적 편의] 보고서를 참고하여 주식 보유 기간을 결정짓는 데에는 시장 구분(코스닥,코스피 등), 보유 주식 종목 수, 시가 총액 규모 등이 유의미한 작용을 하는 것을 알게 되었습니다. (https://www.kcmi.re.kr/publications/publications_list) 

저희는 실제로 이 요인들이 영향력이 있는지 탐구해보면서 분석을 진행하였습니다. 코스피와 코스닥의 경우, 코스닥에 투자한 경우 보유 기간이 길 것으로 예측하였습니다. 그리고 개인이 보유하고 있는 주식 종목 수가 많아 분산 투자를 하고 있다는 예측이 되는 경우, 보유 기간이 짧을 것으로 예상하였습니다. 마지막으로 소형주의 경우 대형주보다는 주식 보유일이 적을 것으로 판단하였습니다. 그러나 데이터를 가공하고 분석에 사용해본 결과 시장구분 칼럼 외의 두 칼럼은 오히려 모델의 성능을 떨어트리는 결과를 가져왔습니다. 따라서 보유 주식 종목 수 , '소형주/소형주 아님'의 이진 분류는 칼럼에서 제외하였습니다.

분석 과정은 1) 탐색적 데이터 분석 2) 전처리 3) 모델 설정 4) 후처리 과정으로 진행하였습니다.

- 01 : 탐색적 데이터 분석 : 주로 히스토그램을 이용하여 데이터들의 분포를 확인하였습니다. 변수 간 상관 관계를 확인하기도 하였습니다.
- 02 : 전처리 : 날짜 데이터의 경우 year, month, 요일을 추가하였습니다. 과거 보유일자 데이터에서 보유 기간을 계산하였습니다.
- 03 : 모델 설정 : LGBM과 StratifiedKFold를 이용하였습니다.
- 04 : 후처리 : 예측값의 상한선과 하한선을 설정하였습니다.



### A'. 채택된 변수에 대한 설명

분석에는 `16개의 변수`를 사용하였습니다.

성별(sex_dit_cd)   
고객자산구간(cus_aet_stn_cd)  
life stage(lsg_sgm_cd)  
서비스등급(tco_cus_grd_cd)  
총 투자기간(mrz_btp_dit_cd)  
주거래업종(mrz_btp_dit_cd)  
투자성향(ivs_icn_cd )  
주식 종목업종(btp_cfc_cd)  
시가총액 규모(mkt_pr_tal_scl_tp_cd)  
과거 보유기간(hist_d)  

10개의 변수는 가공 없이 사용하였습니다.


`파생변수`는 6개가 있습니다. 

코스피,코스닥 구분(Market)    
잔고 금액/수량(stk_p)   
매수 날짜의 년도(year)  
매수 날짜의 월(month)  
매수 날짜의 요일(week)  

그리고 과거 보유 데이터의 보유기간 칼럼 (DIFF) 가 있습니다.  
과거 보유 데이터에는 한 사람의 매수, 전량 매도일만 나와 있는 것이 아니라 전체 거래 내역이 나와있습니다.  
따라서 id, 주식 명 별로 최소 날짜를 구하고 이를 잔고 수량이 0 이 되는 날짜에서 영업일을 카운트하여 구하였습니다.  
주어진 기간 동안에 전량 매도하지 않아 결측이 발생한 행에 대해서는  
2020년 12월 31에서 주식 매수 날짜까지의 영업일을 세어 그 값으로 대체하였습니다.  

제외한 칼럼의 경우에는 다른 변수와 상관관계가 높았던 연령대(cus_age_stn_cd), 대부분이 '국내 주식 투자'에 몰려 있는 주거래상품군(mrz_pdt_tp_sgm_cd), 파생변수를 만드는데 사용한 잔고 수량과 잔고 금액, 그리고 의미가 없다고 판단하였던 액면가(stk_par_pr)를 선택하였습니다.

### B. 모델링 진행

모델은 LGBM regressor를 이용하였고, 이 과정에서 StratifiedKFold를 같이 사용하였습니다.  
하이퍼 파라메터는 optuna와 Gridsearch를 이용하여 찾았습니다.   

최종 모형은 64.4의 결과를 보여주었습니다.

### C. 후처리

hold_d(전체 보유기간) - hist_d(2020.12.31까지의 보유기간) 은 146일 이내입니다.  
따라서 예측값인 hold_d는 hist_d+146을 넘지 않도록 해야 합니다.  
또한 hist_d가 hold_d보다 작을 수는 없습니다.  
따라서 hold_d의 상한선은 hist_d+146, 하한선은 hist_d로 지정하는 방법으로 후처리를 진행하였습니다  

### D. 논의할 점

우선 주말을 제외한 공휴일을 어떻게 지정할지 고민을 많이 했습니다. 그러나 시간 상의 문제로 이를 해결하지 못하고 주말만을 기준으로 영업일을 계산하였다는 점이 후에 보완할 사항입니다.  
LGBM 외 XGBoost나 Catboost 등을 이용하고자 하였으나 모수 튜닝 과정이 시간이 너무 오래 걸려 시도하지 못한 점이 아쉽습니다.


감사합니다.